### Q1.
You are given 1,069 stop records in the file ‘original_with_duplicates_noid.csv’, but  unfortunately technical errors have resulted in some exact duplicate records.  You know  there are around 1,000 unique stops in the file (there may be slightly more or slightly  fewer).   

a) Use a clustering technique of your choice to determine how many unique records  there are.   
b) Check your work by looking for row­wise duplicates using any technique of your  choice (this should be straightforward).  How well did your clustering technique  perform?  
c) Write a paragraph explaining what you did in parts a) and b)  

In [57]:
import pandas as pd
%matplotlib inline
import seaborn as sns
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
import pysal as ps
import mplleaflet
import zipfile 
import sklearn.cluster

In [58]:
data = pd.read_csv('original_with_duplicates_noid.csv')


In [59]:
data1 = pd.DataFrame(columns=data.columns)

In [60]:
data1.append(data[1:2])

,year,found.weapon,found.gun,arrested,suspect.race,suspect.age,suspect.build,suspect.sex,suspect.height,suspect.weight,...,additional.associating,additional.direction,additional.highcrime,additional.time,additional.sights,additional.other,radio.run,day,month,time.period
1,2011,False,False,False,asian,40,medium,male,6.166667,225,...,False,False,True,True,False,False,False,Friday,November,6


In [61]:
data1

,year,found.weapon,found.gun,arrested,suspect.race,suspect.age,suspect.build,suspect.sex,suspect.height,suspect.weight,...,additional.associating,additional.direction,additional.highcrime,additional.time,additional.sights,additional.other,radio.run,day,month,time.period


In [62]:
for elem in ['day','month','location.housing','suspect.sex','suspect.build','suspect.race']:
    

    unique_list = list(set(data[elem]))
    days_dict = dict(zip(unique_list,range(len(unique_list))))
    data[elem] = map(lambda x:days_dict[x],data[elem])


In [64]:
db = sklearn.cluster.DBSCAN(eps=0.5, min_samples=1).fit(data)
core_samples_mask = np.zeros_like(db.labels_, dtype=bool)
core_samples_mask[db.core_sample_indices_] = True
labels = db.labels_

# Number of clusters in labels, ignoring noise if present.
n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)

In [65]:
n_clusters_
#Checking the number of clusters

997

In [66]:
data['label'] = labels
#assigning labels

In [67]:
print len(data.columns)

40


In [68]:
a = np.zeros(shape=(n_clusters_,40))
for i in labels:
    
    a[i,:] = data[data['label']==i][0:1]
    


In [69]:
data1 = pd.DataFrame(a,columns = data.columns)

In [70]:
print len(data1)

997


### Accuracy Measure

In [72]:
data_test = data.drop_duplicates()
data_subset = data1
Percentage = (len(data_subset[data_subset.isin(data_test)])*1.0/len(data_test))*100
print Percentage

100.0


1.I tried different values of eps (The maximum distance between two samples for them to be considered as in the same neighborhood.) and kept the min_sample=1 means I allow one cluster to have one values, which is what we need to get unique rows.
2.The second part just calculates the unique dataframe by dropping the duplicates, and I used that as a measure of finding the value of 'eps'.
The different values of eps that I tried are not shown here, but I used the accuracy measure as a metric to see what value of eps to use till I got the one that was the best.